# Librerias

In [ ]:
import pandas as pd
import numpy as np

#Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import sklearn as sk



#modelos y métricas
import pickle
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score, precision_recall_curve, roc_curve
from sklearn.metrics import confusion_matrix, classification_report
#preprocesamiento
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

#hiperparametros
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer


#configuración warnings
import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Importación del dataset

In [ ]:
url_train = 'https://raw.githubusercontent.com/FrancoSecchi/7506R-1C2023-GRUPO02/checkpoint-2/checkpoint-2/hotels-train.csv'
url_test = 'https://raw.githubusercontent.com/FrancoSecchi/7506R-1C2023-GRUPO02/checkpoint-2/checkpoint-2/hotels-test.csv'
df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)